In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
model_path = "../../corpus/sbert_stair"
# data_name = "hate_labeled.csv"

In [12]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models
sbert = SentenceTransformer(model_path)

In [13]:
def text2vec(text):
    sentences = []
    if isinstance(text, str):
        sentences = [text]
    elif isinstance(text, list):
        sentences = text
    
    return sbert.encode(sentences)


In [14]:
def make_dataset(df, mode="All"):
    X = []
    y = []
    path = "../hand_labeled/"
    datalist = ['DCM', 'DIT', 'IRS']
    convs = read_conv(path, datalist)

    usr_utt = []
    for conv in convs:
        for i, ut in enumerate(conv):
            if not ut.is_system():
                usr_utt.append(clean_text(ut.utt))
    import random

    if mode=="All":
        for la, txt in zip(df.label, df.txt):
            # X.append( InputExample(guid="", texts=[txt], label=float(la) ) )
            X.append(txt)
            y.append(la)
        print("length of X", len(X))
        # 0 の要素を増やしておきますわよ
        sampled = random.sample(usr_utt, len(X)//3)
        for sample in sampled:
            # X.append( InputExample(guid="" , texts=[sample], label=0.0 ) )
            X.append(sample)
            y.append(0)
        print("added length of X", len(X))

    return X, y

In [15]:
corpus_path = "../../corpus/twitter/"
data_name = "hate_labeled.csv"
df = pd.read_csv(corpus_path+data_name)
X, y = make_dataset(df)

length of X 1138
added length of X 1517


In [16]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [17]:
X_train = text2vec(X_train_str)

In [18]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()

In [20]:
clf.fit(X_train, y_train)

AdaBoostClassifier()

In [21]:
print(X_train_str[:5])
print(y_train[:5])

['実際あったら最高に頭悪いマザボだな', 'あまちゃんは商品ではないですよ。', 'たまに画質ゴミゴミになるのなに……', 'それなー、運営も頭悪い', '喧嘩に関しちゃあ……馬鹿じゃねえんだな']
[2, 0, 0, 2, 0]


In [22]:
X_test = text2vec(X_test_str)

In [23]:
y_pred = clf.predict(X_test)

In [33]:
for y_p, x_s in zip(y_pred_[:30], X_test_str[:30]):
    print("{0} : {1}".format(y_p, x_s))

0 : 昨日飲み過ぎて頭悪いの
1 : サムスンは死ね
0 : なんで俺馬鹿にされてるん？
1 : へをこいてはよう死ね
0 : 大塚愛が好きなんですか？
1 : 塾終わったらまた行きたいけど親多分いるあー死ね
1 : マスクゴミ動画量産女たちは韓国の整形美女軍団見習ってほしい
0 : 後払いなら減給じゃなくて草あいつ馬鹿だろ
1 : 死ね！
0 : このメンバーの飲み会馬鹿笑いできるから楽しい
0 : それは良かった。夏ばてしてない？
0 : いいえ組織です。
0 : 大阪といえば、道頓堀が有名ですよね。
0 : 久々に絡みたかったので馬鹿にしてみました
1 : 馬鹿だが?
0 : 天気かな
0 : 本当山口俊って頭悪いんだよね
0 : わからないけど、おいしいよね。
1 : きめえんだよゴミコンクリート流し込んだろか
1 : 死ね沖田
0 : それは知らなかったです。趣味を教えてくれませんか?
0 : 満を待つって何や？
0 : 馬鹿にされちゃう！！！！！！
1 : 馬鹿左翼必死やなぁ
0 : いやいや言い方変えると自分はあの時無謀で馬鹿な人です
0 : 犬は飼ってないんですよね。犬派ですか？猫派ですか？
0 : どちらも昔からの発酵食品で体に良いですよ
1 : んな馬鹿な
0 : ありがとうございます 私達飼い主も親馬鹿ですがかわいいと思ってます
0 : あ、馬鹿だ。ツイ消ししちゃった…馬鹿だ…


In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[150   2   4  11]
 [ 10  14   8  13]
 [  8   6  60  33]
 [  9   1  22 105]]
accuracy =  0.7214912280701754


In [26]:
import copy
def convert_bool(p):
    new_p = copy.deepcopy(p)
    for i, v in enumerate(p):
        if v<=1:
            new_p[i] = 0
        else:
            new_p[i] = 1
    return new_p

In [31]:
y_test_ = convert_bool(y_test)
y_pred_ = convert_bool(y_pred)

In [32]:
print('confusion matrix = \n', confusion_matrix(y_true=y_test_, y_pred=y_pred_))
print('accuracy = ', accuracy_score(y_true=y_test_, y_pred=y_pred_))
print('precision = ', precision_score(y_true=y_test_, y_pred=y_pred_))
print('recall = ', recall_score(y_true=y_test_, y_pred=y_pred_))
print('f1 score = ', f1_score(y_true=y_test_, y_pred=y_pred_))

confusion matrix = 
 [[176  36]
 [ 24 220]]
accuracy =  0.868421052631579
precision =  0.859375
recall =  0.9016393442622951
f1 score =  0.88


In [40]:
tests = ["知らんのかい", "お前馬鹿だろ", "センスないね", "雑魚がよ", "俺が馬鹿だった"]

In [41]:
vec = text2vec(tests)
clf.predict(vec)

array([0, 3, 0, 3, 1])